# F.4 Calculations

In [1]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

### Define Symbols and Configuration Variables

In [2]:
# Time symbol
t = symbols('t')

In [3]:
# Generalized coordinates

# TODO: Change this!
z = symbols('z', cls=Function)
z = z(t)
h = symbols('h', cls=Function)
h = h(t)
theta = symbols('\\theta', cls=Function)
theta = theta(t)

In [4]:
# Derivatives of Generalized Coordinates

# TODO: Change this!
z_dot = z.diff(t)
z_ddot = z_dot.diff(t)
h_dot = h.diff(t)
h_ddot = h_dot.diff(t)
theta_dot = theta.diff(t)
theta_ddot = theta_dot.diff(t)

In [5]:
# Other symbols (mass, forces, damping coefficients, torques...)

# TODO: Change this!
fl, fr, fd, m, mc, d, jc, p0, g, mu = symbols('F_l, F_r, F_d, m, m_c, d, J_c, P_0, g, \\mu', real=True)

### Kinetic Energy

#### Translational Kinetic Energy

In [6]:
# TODO: Change this!
K_T = Rational(1,2)*jc*theta**2 + d**2*m*theta_dot**2 + m*h_dot**2 + m*z_dot**2 + Rational(1,2)*mc*h_dot**2 + Rational(1,2)*mc*z_dot**2

dotprint(K_T)

<IPython.core.display.Math object>

#### Rotational Kinetic Energy

In [7]:
# TODO: Change this!
K_R = 0

dotprint(K_R)

<IPython.core.display.Math object>

#### Total Kinetic Energy

In [8]:
K = K_T #+ K_R

dotprint(K)

<IPython.core.display.Math object>

### Potential Energy

In [9]:
# TODO: Change this!
P = 2*m*g*h + mc*g*h + p0

dotprint(P)

<IPython.core.display.Math object>

### Define the Lagrangian

For this class, the Lagrangian will in general be a scalar value equal to the difference between kinetic and potential energy.

In [10]:
L = K - P

dotprint(L)

<IPython.core.display.Math object>

### Define the Forces

#### Generalized Forces

The forces will form a vector with the same length as the number of configuration variables. 

The first entry corresponds to the first configuration variable, the second entry to the second, etc.

Each entry should contain the generalized forces acting on the corresponding configuration variable.

In [11]:
# TODO: Change this!
tau = Matrix([-fr*sin(theta) -fl*sin(theta), fr*cos(theta) + fl*cos(theta), fr*d - fl*d])

dotprint(tau)

<IPython.core.display.Math object>

#### Non-conservative Forces (e.g. Damping)

Similar to the generalized forces, this will be a vector with the same length as the number of configuration variables.

Each entry should contain the nonconservative forces acting on the corresponding configuration variable.

In [12]:
# TODO: Change this!
B = Matrix([[-mu, 0 , 0], [0, 0, 0], [0, 0, 0]])
q_dot = Matrix([z_dot, 0 , 0])

Bqdot = B @ q_dot

dotprint(Bqdot)

<IPython.core.display.Math object>

In [13]:
F_total = tau + Bqdot

dotprint(F_total)

<IPython.core.display.Math object>

### Compute the Lagrangian Derivatives / Partial Derivatives

The following function computes these two terms:

* $\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right)$
* $\frac{\partial L(q,\dot{q})}{\partial q}$

Things to note:

* `q` and `q_dot` should be Python tuples containing your configuration variables (e.g. `(x, theta)`)
* The function returns a Python list. You probably want to wrap the output in a Sympy `Matrix()`. For example,

```python
out = derive_Lagrangian(...)
out = Matrix(out)
```

In [14]:
def derive_Lagrangian(L, z, h, theta, z_dot, h_dot, theta_dot):
    term_1 = (sympy.tensor.derive_by_array(L, z_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, z)
    term_3 = (sympy.tensor.derive_by_array(L, h_dot)).diff(t)
    term_4 = sympy.tensor.derive_by_array(L, h)
    term_5 = (sympy.tensor.derive_by_array(L, theta_dot)).diff(t)
    term_6 = sympy.tensor.derive_by_array(L, theta)
    
    ans = Matrix([term_1 - term_2, term_3 - term_4, term_5 - term_6])
    return ans

### Get the final Euler Lagrange Equations of Motion

Remember that the Euler Lagrange Equations used in class are:

$$
\underbrace{\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right) - \frac{\partial L(q,\dot{q})}{\partial q}}_\text{Left Hand Side (LHS)} = \underbrace{\tau - B \dot{q}}_\text{Right Hand Side (RHS)}.
$$

In Sympy, we use the `Eq(LHS, RHS)` class to get a symbolic equation $\text{LHS} = \text{RHS}$.

Both the left and right expressions should be Sympy vectors (i.e. Matrices with 1 column). 

The length of the left and right vectors must both be equal to the number of configuration variables.

In [15]:
# TODO: Change this!
LHS = derive_Lagrangian(L, z, h, theta, z_dot, h_dot, theta_dot)
RHS = F_total

Euler_Lagrange = Eq(LHS, RHS)

dotprint(simplify(Euler_Lagrange))

<IPython.core.display.Math object>

# linearize

In [16]:
solve_dict = solve(Euler_Lagrange, (z_ddot, h_ddot, theta_ddot), simplify = True, dict = True)[0]
dotprint(solve_dict)

<IPython.core.display.Math object>

In [21]:
state = Matrix([z, z_dot, h, h_dot, theta, theta_dot])

In [22]:
z_ddot_expr = solve_dict[z_ddot]
h_ddot_expr = solve_dict[h_ddot]
theta_ddot_expr = solve_dict[theta_ddot]
state_deriv = Matrix([z_dot, z_ddot_expr, h_dot, h_ddot_expr, theta_dot, theta_ddot_expr])
dotprint(state_deriv)

<IPython.core.display.Math object>

In [23]:
A = state_deriv.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

In [25]:
inputs = Matrix([[tau]])

In [26]:
B = state_deriv.jacobian(inputs)
dotprint(B)

ValueError: 
Can't calculate derivative wrt -F_l*sin(\theta(t)) -
F_r*sin(\theta(t)).